# IFN695 - Report Code

## Importing all Relevant Libraries

In [4]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import plotly.graph_objects as go

from report_utils import read_file, log_time, format_dataframe, merge_duids, split_region
import inspect
import time

from nemosis import dynamic_data_compiler

pd.set_option('display.max_rows', 20)

## Importing Datasets

In [5]:
renewable_penetration = pd.read_csv('data/NEM_RENEWABLE_PENETRATION_ALL_202508041827.csv')
renewable_penetration

,DateTime,Max/Min,State,Fuel Type,Supply
0,2/1/2018 4:00,Min,NEM,Battery,0.000
1,2/1/2018 4:00,Min,NEM,Biomass,10.233
2,2/1/2018 4:00,Min,NEM,Black coal,10655.808
3,2/1/2018 4:00,Min,NEM,Brown coal,4655.597
4,2/1/2018 4:00,Min,NEM,Distributed PV,0.000
...,...,...,...,...,...
1858,3/8/2025 11:30,Max,NEM,Gas,258.971
1859,3/8/2025 11:30,Max,NEM,Hydro,586.385
1860,3/8/2025 11:30,Max,NEM,Liquid Fuel,0.000
1861,3/8/2025 11:30,Max,NEM,Utility-scale Solar,5725.921


## Loading the Archived Packages for Fuel Mix in Each State

In [6]:
start_date = '2018/01/01 00:00:00'
end_date = '2025/07/31 00:00:00'

current_dir = os.getcwd()
fuel_raw_data_cache = os.path.join(current_dir, 'data', 'fuel_mix')
price_raw_data_cache = os.path.join(current_dir, 'data', 'price')

# dudetail = dynamic_data_compiler(
#     start_time = start_date,
#     end_time = end_date,
#     raw_data_location = fuel_raw_data_cache, 
#     table_name = 'DUDETAILSUMMARY',
#     fformat = 'csv'
# )

# dispatch = dynamic_data_compiler(
#     start_time = start_date,
#     end_time = end_date,
#     raw_data_location = fuel_raw_data_cache,
#     table_name = 'DISPATCH_UNIT_SCADA',
#     fformat = 'csv'
# )

# dispatch_price = dynamic_data_compiler(
#     start_time = start_date,
#     end_time = end_date,
#     raw_data_location = price_raw_data_cache,
#     table_name = 'DISPATCHPRICE',
#     fformat = 'csv'
# )

In [7]:
dudetail_data = pd.read_csv('data/fuel_mix/PUBLIC_ARCHIVE#DUDETAILSUMMARY#FILE01#202507010000.CSV', header = 1)
dudetail_generator_data = dudetail_data[dudetail_data['DISPATCHTYPE'] == 'GENERATOR']
dudetail_generator_data

,I,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7,DUID,START_DATE,END_DATE,DISPATCHTYPE,CONNECTIONPOINTID,REGIONID,...,IS_AGGREGATED,DISPATCHSUBTYPE,ADG_ID,LOAD_MINIMUM_ENERGY_PRICE,LOAD_MAXIMUM_ENERGY_PRICE,LOAD_MIN_RAMP_RATE_UP,LOAD_MIN_RAMP_RATE_DOWN,LOAD_MAX_RAMP_RATE_UP,LOAD_MAX_RAMP_RATE_DOWN,SECONDARY_TLF
0,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,LK_ECHO,2019/10/10 00:00:00,2020/07/01 00:00:00,GENERATOR,TLE11,TAS1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,LK_ECHO,2020/12/15 00:00:00,2021/07/01 00:00:00,GENERATOR,TLE11,TAS1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,LK_ECHO,2024/06/04 00:00:00,2024/07/01 00:00:00,GENERATOR,TLE11,TAS1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,LNGS1,2016/11/01 00:00:00,2017/07/01 00:00:00,GENERATOR,VAT21L,VIC1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,LNGS1,2021/12/20 00:00:00,2022/07/01 00:00:00,GENERATOR,VAT21L,VIC1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22120,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,LRSF1,2025/07/22 00:00:00,2999/12/31 00:00:00,GENERATOR,QLLV2L,QLD1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22121,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,SHEP1,2025/07/22 00:00:00,2999/12/31 00:00:00,GENERATOR,VSHT2S,VIC1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22122,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,TATURA01,2025/07/22 00:00:00,2999/12/31 00:00:00,GENERATOR,VSHT1,VIC1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22123,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,VALDORA1,2025/07/22 00:00:00,2999/12/31 00:00:00,GENERATOR,QPWD1S,QLD1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Function Used to Split Datasets into State Level Dispatch
The workflow:
- Get inside the directory (public/archived)
- Read each file
Within each file:
- Format File into a monthly `strftime`
- Merge with the `duid_regions` - to get the region for each generator
- Split the data into each region as a separate dataframe

In [8]:
def populate_state_level_datasets(generator_dirs):
    # Main, Monthly DataFrames
    nsw_data = pd.DataFrame()
    qld_data = pd.DataFrame()
    sa_data = pd.DataFrame()
    tas_data = pd.DataFrame()
    vic_data = pd.DataFrame()

    state_data_columns = ['SETTLEMENTDATE', 'DUID', 'SCADAVALUE']

    for dir in generator_dirs:
        print('Inside directory: ', dir)

        # This picks up the dispatch electricity folders
        dispatch_folder = sorted(os.listdir(dir))
        try:
            dispatch_folder.remove('.DS_Store')
        except:
            pass

        for file in sorted(os.listdir(dir)):
            if file[0:6] == 'PUBLIC':
                file_path = os.path.join(dir, file)

                print(f'Reading File: {file}')
                monthly_data = log_time("Read File", read_file, file_path, state_data_columns)
                monthly_data = log_time("Dates Formatted", format_dataframe, monthly_data, "SETTLEMENTDATE", True, True)

                region_data = log_time('Regions Merged', merge_duids, monthly_data)

                # The main dataset
                nsw_data = log_time(f"Split Region NSW", split_region, nsw_data, region_data, 'NSW1')
                qld_data = log_time(f"Split Region QLD", split_region, qld_data, region_data, 'QLD1')
                sa_data = log_time(f"Split Region SA", split_region, sa_data, region_data, 'SA1')
                tas_data = log_time(f"Split Region TAS", split_region, tas_data, region_data, 'TAS1')
                vic_data = log_time(f"Split Region VIC", split_region, vic_data, region_data, 'VIC1')

    return nsw_data, qld_data, sa_data, tas_data, vic_data

In [9]:
public_fuel_dir = os.path.join(current_dir, 'data', 'fuel_mix', 'public')
archived_fuel_dir = os.path.join(current_dir, 'data', 'fuel_mix', 'archived')

public_price_dir = os.path.join(current_dir, 'data', 'price', 'public')
archived_price_dir = os.path.join(current_dir, 'data', 'price', 'archived')

In [10]:
nsw_data, qld_data, sa_data, tas_data, vic_data = populate_state_level_datasets([public_fuel_dir, archived_fuel_dir])

Inside directory:  /Users/aidanlockwood/Documents/GitHub/IFN695-Codebase/data/fuel_mix/public
Reading File: PUBLIC_DVD_DISPATCH_UNIT_SCADA_201801010000.CSV
Read File in 1.35 seconds
Dates Formatted in 0.6 seconds
Regions Merged in 0.06 seconds
Split Region NSW in 0.16 seconds
Split Region QLD in 0.15 seconds
Split Region SA in 0.14 seconds
Split Region TAS in 0.14 seconds
Split Region VIC in 0.16 seconds
Reading File: PUBLIC_DVD_DISPATCH_UNIT_SCADA_201802010000.CSV
Read File in 1.38 seconds
Dates Formatted in 0.49 seconds
Regions Merged in 0.05 seconds
Split Region NSW in 0.21 seconds
Split Region QLD in 0.17 seconds
Split Region SA in 0.16 seconds
Split Region TAS in 0.14 seconds
Split Region VIC in 0.18 seconds
Reading File: PUBLIC_DVD_DISPATCH_UNIT_SCADA_201803010000.CSV
Read File in 1.07 seconds
Dates Formatted in 0.57 seconds
Regions Merged in 0.06 seconds
Split Region NSW in 0.18 seconds
Split Region QLD in 0.19 seconds
Split Region SA in 0.18 seconds
Split Region TAS in 0.15 sec

## Loading in the Generator Information to Connect the DUIDs to Fuel Mix 


In [11]:
generator_information = pd.read_excel('data/NEM Generation Information July 2025.xlsx', sheet_name = 'ExistingGeneration&NewDevs', header = 1).dropna(subset = ['DUID'])

qld_generators = generator_information[generator_information['Region'] == 'QLD1']
nsw_generators = generator_information[generator_information['Region'] == 'NSW1']
vic_generators = generator_information[generator_information['Region'] == 'VIC1']
tas_generators = generator_information[generator_information['Region'] == 'TAS1']
sa_generators = generator_information[generator_information['Region'] == 'SA1']

state_generators = [nsw_generators, qld_generators, sa_generators, tas_generators, vic_generators]

Mapping the Generator Data to a DUID to allow for the Fuel Types to be Associated to the Dispatch Data

In [12]:
nem_datasets = [nsw_data, qld_data, sa_data, tas_data, vic_data]

for dataset in state_generators:
    dataset.reset_index(drop = False, inplace = True)

for i, dataset in enumerate(state_generators):
    state_generator_duids = dataset.set_index('DUID', inplace = True)

    state_duid_fuel_types = dataset['Fuel Type'].dropna().to_dict()
    nem_datasets[i]['Fuel Type'] = nem_datasets[i]['DUID'].map(state_duid_fuel_types)


In [13]:
qld_generator_duids = qld_generators.reset_index(drop = False, inplace = True)
qld_generator_duids = qld_generators.set_index('DUID', inplace = True)
qld_duid_fuel_types = qld_generators['Fuel Type'].dropna().to_dict()
qld_data['Fuel Type'] = qld_data['DUID'].map(qld_duid_fuel_types)

In [14]:
for dataset in nem_datasets:
    dataset.reset_index(drop = False, inplace = True)

In [15]:
qld_data.reset_index(drop = False, inplace = True)

In [16]:
qld_data = qld_data.drop(columns = 'index')

In [17]:
qld_data = qld_data[qld_data['MONTH'] != '2025-08']
nsw_data = nsw_data[nsw_data['MONTH'] != '2025-08']
sa_data = sa_data[sa_data['MONTH'] != '2025-08']  
tas_data = tas_data[tas_data['MONTH'] != '2025-08']
vic_data = vic_data[vic_data['MONTH'] != '2025-08']

In [18]:
qld_data.drop(columns = 'level_0', inplace = True)
nsw_data.drop(columns = 'index', inplace = True)
sa_data.drop(columns = 'index', inplace = True)
tas_data.drop(columns = 'index', inplace = True)
vic_data.drop(columns = 'index', inplace = True)

/var/folders/h9/_mkwm6vx2qjf9s5hb6rlyg8c0000gn/T/ipykernel_34356/2741488734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nsw_data.drop(columns = 'index', inplace = True)
/var/folders/h9/_mkwm6vx2qjf9s5hb6rlyg8c0000gn/T/ipykernel_34356/2741488734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa_data.drop(columns = 'index', inplace = True)
/var/folders/h9/_mkwm6vx2qjf9s5hb6rlyg8c0000gn/T/ipykernel_34356/2741488734.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [19]:
nsw_data.columns = ['SETTLEMENTDATE',  'DUID', 'SCADAVALUE', 'MONTH', 'REGIONID', 'Fuel Type']
sa_data.columns = ['SETTLEMENTDATE', 'DUID', 'SCADAVALUE', 'MONTH', 'REGIONID', 'Fuel Type']
tas_data.columns = ['SETTLEMENTDATE', 'DUID', 'SCADAVALUE', 'MONTH', 'REGIONID', 'Fuel Type']
qld_data.columns = ['SETTLEMENTDATE', 'DUID', 'SCADAVALUE', 'MONTH', 'REGIONID', 'Fuel Type']
vic_data.columns = ['SETTLEMENTDATE', 'DUID', 'SCADAVALUE', 'MONTH', 'REGIONID', 'Fuel Type']

In [20]:
qld_fuel_generation_agg = qld_data.groupby(['MONTH', 'Fuel Type'])['SCADAVALUE'].sum().reset_index()
nsw_fuel_generation_agg = nsw_data.groupby(['MONTH', 'Fuel Type'])['SCADAVALUE'].sum().reset_index()
sa_fuel_generation_agg = sa_data.groupby(['MONTH', 'Fuel Type'])['SCADAVALUE'].sum().reset_index()
tas_fuel_generation_agg = tas_data.groupby(['MONTH', 'Fuel Type'])['SCADAVALUE'].sum().reset_index()
vic_fuel_generation_agg = vic_data.groupby(['MONTH', 'Fuel Type'])['SCADAVALUE'].sum().reset_index()


Removing the "Other" Fuel Type from the dataset. Not meaningful

In [21]:
qld_fuel_generation_agg = qld_fuel_generation_agg[qld_fuel_generation_agg['Fuel Type'] != 'Other - Other']
nsw_fuel_generation_agg = nsw_fuel_generation_agg[nsw_fuel_generation_agg['Fuel Type'] != 'Other - Other']
vic_fuel_generation_agg = vic_fuel_generation_agg[vic_fuel_generation_agg['Fuel Type'] != 'Other - Other']
sa_fuel_generation_agg = sa_fuel_generation_agg[sa_fuel_generation_agg['Fuel Type'] != 'Other - Other']

Due to an issue when visualising the above datasets, now need to convert the months back to datetime

In [22]:
qld_fuel_generation_agg['MONTH'] = qld_fuel_generation_agg['MONTH'].dt.to_timestamp()
nsw_fuel_generation_agg['MONTH'] = nsw_fuel_generation_agg['MONTH'].dt.to_timestamp()
sa_fuel_generation_agg['MONTH'] = sa_fuel_generation_agg['MONTH'].dt.to_timestamp()
tas_fuel_generation_agg['MONTH'] = tas_fuel_generation_agg['MONTH'].dt.to_timestamp()
vic_fuel_generation_agg['MONTH'] = vic_fuel_generation_agg['MONTH'].dt.to_timestamp()

In [23]:
qld_fuel_generation_agg['MONTH'] = qld_fuel_generation_agg['MONTH'].dt.strftime('%m-%Y')
nsw_fuel_generation_agg['MONTH'] = nsw_fuel_generation_agg['MONTH'].dt.strftime('%m-%Y')
sa_fuel_generation_agg['MONTH'] = sa_fuel_generation_agg['MONTH'].dt.strftime('%m-%Y')
tas_fuel_generation_agg['MONTH'] = tas_fuel_generation_agg['MONTH'].dt.strftime('%m-%Y')
vic_fuel_generation_agg['MONTH'] = vic_fuel_generation_agg['MONTH'].dt.strftime('%m-%Y')

There is an 'Other' value for Fuel Type. Going to remove that for easier analysis

In [24]:
renewable_penetration = renewable_penetration[renewable_penetration['Fuel Type'] != 'Other']
renewable_penetration

,DateTime,Max/Min,State,Fuel Type,Supply
0,2/1/2018 4:00,Min,NEM,Battery,0.000
1,2/1/2018 4:00,Min,NEM,Biomass,10.233
2,2/1/2018 4:00,Min,NEM,Black coal,10655.808
3,2/1/2018 4:00,Min,NEM,Brown coal,4655.597
4,2/1/2018 4:00,Min,NEM,Distributed PV,0.000
...,...,...,...,...,...
1858,3/8/2025 11:30,Max,NEM,Gas,258.971
1859,3/8/2025 11:30,Max,NEM,Hydro,586.385
1860,3/8/2025 11:30,Max,NEM,Liquid Fuel,0.000
1861,3/8/2025 11:30,Max,NEM,Utility-scale Solar,5725.921


The Supply column is the only column that has the correct format. Going to change the:
- DateTime to a Date Object - as this will be a monthly average, min and max 
- FuelType to a category 
- Max/Min transposed into their own variables with a calculated monthly average

Converting the Date Column

In [25]:
# Converting the DateTime column
renewable_penetration['DateTime'] = pd.to_datetime(renewable_penetration['DateTime'], format='mixed', dayfirst=True)
renewable_penetration['DateTime'] = renewable_penetration['DateTime'].dt.strftime('%Y-%m')

/var/folders/h9/_mkwm6vx2qjf9s5hb6rlyg8c0000gn/T/ipykernel_34356/426761549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_penetration['DateTime'] = pd.to_datetime(renewable_penetration['DateTime'], format='mixed', dayfirst=True)
/var/folders/h9/_mkwm6vx2qjf9s5hb6rlyg8c0000gn/T/ipykernel_34356/426761549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_penetration['DateTime'] = renewable_penetration['DateTime'].dt.strftime('%Y-%m')


### Creating the Groupby's to Calculate the Min, Max and Mean Supply Amounts

In [26]:
mean_supply = renewable_penetration.groupby(['Fuel Type', 'DateTime'])['Supply'].mean().reset_index()
min_supply = renewable_penetration.groupby(['Fuel Type', 'DateTime'])['Supply'].min().reset_index()['Supply']
max_supply = renewable_penetration.groupby(['Fuel Type', 'DateTime'])['Supply'].max().reset_index()['Supply']

# Adding the min and max supplies to the mean supply groupby
mean_supply['Min Supply (MW)'] = min_supply
mean_supply['Max Supply (MW)'] = max_supply

In [27]:
mean_supply.columns = ['Fuel Type', 'Month', 'Mean Supply (MW)', 'Min Supply (MW)', 'Max Supply (MW)']
mean_supply

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Battery,2018-02,0.6665,0.000,1.333
2,Battery,2018-03,2.8665,0.000,5.733
3,Battery,2018-04,4.8250,0.000,9.650
4,Battery,2018-05,3.0330,2.550,3.516
...,...,...,...,...,...
915,Wind,2025-04,2027.1265,1279.964,2774.289
916,Wind,2025-05,3007.9465,1232.864,4783.029
917,Wind,2025-06,3260.6030,1857.128,4664.078
918,Wind,2025-07,3395.3705,1422.093,5368.648


Generating the new dataframe with the supplies, sorted by months

In [28]:
supply_summary = mean_supply.sort_values(by = 'Month').reset_index(drop = True)
supply_summary

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Liquid Fuel,2018-01,0.0595,0.021,0.098
2,Black coal,2018-01,12383.7065,10655.808,14111.605
3,Hydro,2018-01,1122.2300,355.826,1888.634
4,Biomass,2018-01,12.3745,10.233,14.516
...,...,...,...,...,...
915,Black coal,2025-08,9114.7900,6399.196,11830.384
916,Biomass,2025-08,40.5620,35.100,46.024
917,Battery,2025-08,7.2425,2.324,12.161
918,Utility-scale Solar,2025-08,2863.1915,0.462,5725.921


In [29]:
battery_summary = supply_summary[supply_summary['Fuel Type'] == 'Battery']

In [30]:
supply_summary[supply_summary['Month'] == '2018-01']

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Liquid Fuel,2018-01,0.0595,0.021,0.098
2,Black coal,2018-01,12383.7065,10655.808,14111.605
3,Hydro,2018-01,1122.2300,355.826,1888.634
4,Biomass,2018-01,12.3745,10.233,14.516
5,Wind,2018-01,1715.5270,566.570,2864.484
6,Brown coal,2018-01,4597.9545,4540.312,4655.597
7,Gas,2018-01,1712.7820,1517.622,1907.942
8,Distributed PV,2018-01,1938.2870,0.000,3876.574
9,Utility-scale Solar,2018-01,134.7040,2.204,267.204


In [31]:
supply_summary['Fuel Type'].unique()

array(['Battery', 'Liquid Fuel', 'Black coal', 'Hydro', 'Biomass', 'Wind',
       'Brown coal', 'Gas', 'Distributed PV', 'Utility-scale Solar'],
      dtype=object)

In [32]:
mean_supply_amounts = supply_summary.groupby(['Month'])['Mean Supply (MW)'].sum().reset_index()['Mean Supply (MW)']

### Creating a DataFrame to split Renewables and Non-Renewables

In [33]:
non_renewables = ['Liquid Fuel', 'Brown coal', 'Black coal', 'Gas']

renewables = supply_summary['Fuel Type'].unique().tolist()

for fuel in non_renewables:
    renewables.remove(fuel)

renewables_df = supply_summary[supply_summary['Fuel Type'].isin(renewables)].reset_index(drop = True)
non_renewables_df = supply_summary[supply_summary['Fuel Type'].isin(non_renewables)].reset_index(drop = True)

In [34]:
renewables_df

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Hydro,2018-01,1122.2300,355.826,1888.634
2,Biomass,2018-01,12.3745,10.233,14.516
3,Wind,2018-01,1715.5270,566.570,2864.484
4,Distributed PV,2018-01,1938.2870,0.000,3876.574
...,...,...,...,...,...
547,Hydro,2025-08,1224.0985,586.385,1861.812
548,Biomass,2025-08,40.5620,35.100,46.024
549,Battery,2025-08,7.2425,2.324,12.161
550,Utility-scale Solar,2025-08,2863.1915,0.462,5725.921


In [35]:
renewables_df[0:7]

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Hydro,2018-01,1122.2300,355.826,1888.634
2,Biomass,2018-01,12.3745,10.233,14.516
3,Wind,2018-01,1715.5270,566.570,2864.484
4,Distributed PV,2018-01,1938.2870,0.000,3876.574
5,Utility-scale Solar,2018-01,134.7040,2.204,267.204
6,Wind,2018-02,1837.8105,437.997,3237.624


### Creating a DataFrame to Aggregate the Mean Supplies of all Fuel Sources
Then plot the total mean supply over time

In [36]:
renewables_supply = renewables_df.groupby(['Month'])['Mean Supply (MW)'].sum().reset_index()
renewables_supply

,Month,Mean Supply (MW)
0,2018-01,4923.4055
1,2018-02,4667.5965
2,2018-03,4460.9820
3,2018-04,4512.6620
4,2018-05,4857.1565
...,...,...
87,2025-04,10536.8795
88,2025-05,10704.8115
89,2025-06,11253.8470
90,2025-07,12593.5200


In [37]:
non_renewables_supply = non_renewables_df.groupby(['Month'])['Mean Supply (MW)'].sum().reset_index()
non_renewables_supply

,Month,Mean Supply (MW)
0,2018-01,18694.5025
1,2018-02,18829.0080
2,2018-03,17066.9565
3,2018-04,16415.0150
4,2018-05,17820.6030
...,...,...
87,2025-04,12175.0755
88,2025-05,12536.7005
89,2025-06,13385.9000
90,2025-07,15253.6980


In [38]:
total_supply_df = pd.merge(renewables_supply, non_renewables_supply, on='Month', suffixes=(' Renewables', ' Non Renewables'))
total_supply_df

,Month,Mean Supply (MW) Renewables,Mean Supply (MW) Non Renewables
0,2018-01,4923.4055,18694.5025
1,2018-02,4667.5965,18829.0080
2,2018-03,4460.9820,17066.9565
3,2018-04,4512.6620,16415.0150
4,2018-05,4857.1565,17820.6030
...,...,...,...
87,2025-04,10536.8795,12175.0755
88,2025-05,10704.8115,12536.7005
89,2025-06,11253.8470,13385.9000
90,2025-07,12593.5200,15253.6980


### Generating the Plots for the Total Fuel Source Renewables/Non-Renewables

In [39]:
fig = px.line(total_supply_df, x='Month', y=['Mean Supply (MW) Renewables', 'Mean Supply (MW) Non Renewables'], title='Total Mean Supply of Non-Renewables vs Renewables in the NEM (2018 - 2025)', labels={'value': 'Mean Supply (MW)', 'variable': 'Fuel Type'})
fig.update_traces(mode = 'lines+markers')

### Generating the Plot for Renewables

In [40]:
px.line(renewables_df, x = 'Month', y = 'Mean Supply (MW)', color = 'Fuel Type', title = 'Mean Monthly Supply of Renewable Energy Sources (2018-2025)')

### Generating the Plot for Non-Renewables

In [41]:
px.line(non_renewables_df, x = 'Month', y = 'Mean Supply (MW)', color = 'Fuel Type', title = 'Mean Monthly Supply of Non-Renewable Energy Sources (2018-2025)')

### Getting the Mean Supplies for Each Renewable Fuel Source in October 2023 and September 2024

In [42]:
october_2023 = renewables_df[renewables_df['Month'] == '2023-10'].sort_values(by = 'Fuel Type').reset_index(drop=True)
october_2023

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2023-10,18.4530,9.029,27.877
1,Biomass,2023-10,66.5195,62.199,70.840
2,Distributed PV,2023-10,5842.3330,0.000,11684.666
3,Hydro,2023-10,1125.4695,445.747,1805.192
4,Utility-scale Solar,2023-10,2356.6765,0.620,4712.733
5,Wind,2023-10,2514.8595,1579.012,3450.707


In [43]:
september_2024 = renewables_df[renewables_df['Month'] == '2024-09'].sort_values(by = 'Fuel Type').reset_index(drop=True)
september_2024

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2024-09,-83.5645,-145.975,-21.154
1,Biomass,2024-09,80.9705,76.527,85.414
2,Distributed PV,2024-09,5417.1240,0.000,10834.248
3,Hydro,2024-09,947.0640,481.970,1412.158
4,Utility-scale Solar,2024-09,2582.4230,0.027,5164.819
5,Wind,2024-09,2992.2475,2204.795,3779.700


In [44]:
october_september_mean_supplies = pd.DataFrame({
    'Fuel Type': october_2023['Fuel Type'],
    'Mean Supply October 2023 (MW)': october_2023['Mean Supply (MW)'],
    'Mean Supply September 2024 (MW)': september_2024['Mean Supply (MW)']
})
october_september_mean_supplies

,Fuel Type,Mean Supply October 2023 (MW),Mean Supply September 2024 (MW)
0,Battery,18.4530,-83.5645
1,Biomass,66.5195,80.9705
2,Distributed PV,5842.3330,5417.1240
3,Hydro,1125.4695,947.0640
4,Utility-scale Solar,2356.6765,2582.4230
5,Wind,2514.8595,2992.2475


### Producing the Minimum Negative Supply Amounts and their Frequency

In [45]:
min_neg_supply = supply_summary[supply_summary['Min Supply (MW)'] < 0].groupby('Fuel Type').min().drop(columns = ['Mean Supply (MW)', 'Max Supply (MW)'])

In [46]:
min_neg_freq = supply_summary[supply_summary['Min Supply (MW)'] < 0]['Fuel Type'].value_counts().to_list()
min_neg_supply['Frequency of Negative Supply Months'] = min_neg_freq
min_neg_supply

,Month,Min Supply (MW),Frequency of Negative Supply Months
Fuel Type,,,
Battery,2024-05,-161.050,15
Biomass,2022-10,-4.950,6
Liquid Fuel,2022-10,-0.100,5
Utility-scale Solar,2022-08,-0.202,4


Cell used to export the data to Excel

In [47]:
min_neg_supply.to_excel('data/min_negative_supplies.xlsx')

### Function to Produce DataFrame for the CAGR of each Source

In [48]:
renewables_df

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Hydro,2018-01,1122.2300,355.826,1888.634
2,Biomass,2018-01,12.3745,10.233,14.516
3,Wind,2018-01,1715.5270,566.570,2864.484
4,Distributed PV,2018-01,1938.2870,0.000,3876.574
...,...,...,...,...,...
547,Hydro,2025-08,1224.0985,586.385,1861.812
548,Biomass,2025-08,40.5620,35.100,46.024
549,Battery,2025-08,7.2425,2.324,12.161
550,Utility-scale Solar,2025-08,2863.1915,0.462,5725.921


In [49]:
def produce_cagr_dataframe(data, start_date, end_date, num_years = 7):

    fuel_sources = data['Fuel Type'].unique().tolist()

    fuel_data = {}

    print(fuel_sources)

    for fuel in fuel_sources:
        start_mean_supply = data[(data['Fuel Type'] == fuel) &(data['Month'] == start_date)]['Mean Supply (MW)'].values[0]
        end_mean_supply = data[(data['Fuel Type'] == fuel) & (data['Month'] == end_date)]['Mean Supply (MW)'].values[0]

        cagr = (((end_mean_supply / start_mean_supply) ** (1 / num_years)) - 1) * 100

        fuel_data[fuel] = [cagr, start_mean_supply, end_mean_supply]

    return pd.DataFrame(fuel_data, index = ['CAGR (%)', f'Start Mean Supply (MW)', f'End Mean Supply (MW)']).T

In [50]:
renewables_cagr = produce_cagr_dataframe(renewables_df, '2018-01', '2025-08')
renewables_cagr

['Battery', 'Hydro', 'Biomass', 'Wind', 'Distributed PV', 'Utility-scale Solar']


,CAGR (%),Start Mean Supply (MW),End Mean Supply (MW)
Battery,58.912273,0.2830,7.2425
Hydro,1.248976,1122.2300,1224.0985
Biomass,18.482977,12.3745,40.5620
Wind,7.981008,1715.5270,2936.4945
Distributed PV,14.618222,1938.2870,5037.2530
Utility-scale Solar,54.752815,134.7040,2863.1915


In [51]:
non_renewables_cagr = produce_cagr_dataframe(non_renewables_df, '2018-01', '2025-08')
non_renewables_cagr

['Liquid Fuel', 'Black coal', 'Brown coal', 'Gas']


,CAGR (%),Start Mean Supply (MW),End Mean Supply (MW)
Liquid Fuel,-100.000000,0.0595,0.0000
Black coal,-4.283867,12383.7065,9114.7900
Brown coal,-3.237220,4597.9545,3651.9350
Gas,-8.579342,1712.7820,914.1395


### Writing the data to an Excel File

In [52]:
renewables_cagr.to_excel('data/renewables_cagr.xlsx')
non_renewables_cagr.to_excel('data/non_renewables_cagr.xlsx')

## Creating the State Level Fuel Type Charts

In [53]:
nsw_fuel_generation_agg_no_coal = nsw_fuel_generation_agg[nsw_fuel_generation_agg['Fuel Type'] != 'Fossil - Black Coal']

In [54]:
px.line(nsw_fuel_generation_agg, x = 'MONTH', y = 'SCADAVALUE', color = 'Fuel Type', title = 'NSW Fuel Type Generation (Jan 2018 - July 2025)', labels = {'SCADAVALUE': 'Generation (MWh)', 'MONTH': 'Month'})

In [55]:
px.line(qld_fuel_generation_agg, x = 'MONTH', y = 'SCADAVALUE', color = 'Fuel Type', title = 'Queensland Fuel Generation by Type (Jan 2018 - July 2025)')

In [56]:
px.line(tas_fuel_generation_agg, x = 'MONTH', y = 'SCADAVALUE', color = 'Fuel Type', title = 'Tasmania Fuel Generation by Type (Jan 2018 - July 2025)')

In [57]:
px.line(vic_fuel_generation_agg, x = 'MONTH', y = 'SCADAVALUE', color = 'Fuel Type', title = 'Victoria Fuel Generation by Type (Jan 2018 - July 2025)')

In [58]:
px.line(sa_fuel_generation_agg, x = 'MONTH', y = 'SCADAVALUE', color = 'Fuel Type', title = 'South Australia Fuel Generation by Type (Jan 2018 - July 2025)')

In [59]:
nsw_fuel_generation_agg.to_excel('data/nsw_fuel_generation_agg.xlsx')

Now we know the total amounts for each fuel, time to combine them into Renewable/Non Renewable

In [60]:
def generate_renewable_datasets(agg_data):
    renewables = []
    non_renewables = []

    for fuel in agg_data['Fuel Type'].unique().tolist():
        if 'Fossil' in fuel:
            non_renewables.append(fuel)
        else:
            renewables.append(fuel)

    renewables_df = agg_data[agg_data['Fuel Type'].isin(renewables)].reset_index(drop = True)
    non_renewables_df = agg_data[agg_data['Fuel Type'].isin(non_renewables)].reset_index(drop = True)
    
    

    renewables_df = renewables_df.reset_index(drop = True)

    return renewables_df, non_renewables_df

In [61]:
nsw_renewables_df, nsw_non_renewables_df = generate_renewable_datasets(nsw_fuel_generation_agg)
qld_renewables_df, qld_non_renewables_df = generate_renewable_datasets(qld_fuel_generation_agg)
vic_renewables_df, vic_non_renewables_df = generate_renewable_datasets(vic_fuel_generation_agg)
sa_renewables_df, sa_non_renewables_df = generate_renewable_datasets(sa_fuel_generation_agg)
tas_renewables_df, tas_non_renewables_df = generate_renewable_datasets(tas_fuel_generation_agg)

In [62]:
nsw_renewables_df['Fuel Type'].unique()

array(['Hydro - Water', 'Solar - Solar', 'Wind - Wind',
       'Renewable Biomass or Waste - Bagasse',
       'Renewable Biomass or Waste - Biomass'], dtype=object)

Things to look at with report:
- Extracting patterns and behaviours of generation 
- Looking at documents and reports for certain policies and behaviours in renewables
- Trying to understand Negative supply values found

## Extracting the State Level Generation Patterns

In [63]:
def format_state_data_sources(renewables, non_renewables):
    renewables['MONTH'] = pd.to_datetime(renewables['MONTH'], format='%m-%Y')
    non_renewables['MONTH'] = pd.to_datetime(non_renewables['MONTH'], format='%m-%Y')

    renewables = renewables.groupby('MONTH')['SCADAVALUE'].sum().reset_index()
    non_renewables = non_renewables.groupby('MONTH')['SCADAVALUE'].sum().reset_index()

    return renewables, non_renewables

qld_renewables_df, qld_non_renewables_df = format_state_data_sources(qld_renewables_df, qld_non_renewables_df)
nsw_renewables_df, nsw_non_renewables_df = format_state_data_sources(nsw_renewables_df, nsw_non_renewables_df)
vic_renewables_df, vic_non_renewables_df = format_state_data_sources(vic_renewables_df, vic_non_renewables_df)
sa_renewables_df, sa_non_renewables_df = format_state_data_sources(sa_renewables_df, sa_non_renewables_df)
tas_renewables_df, tas_non_renewables_df = format_state_data_sources(tas_renewables_df, tas_non_renewables_df)

In [64]:
fig = go.Figure()

fig = fig.add_trace(go.Scatter(x=nsw_renewables_df['MONTH'], y=nsw_renewables_df['SCADAVALUE'], mode='lines+markers', name='NSW Renewables'))
fig = fig.add_trace(go.Scatter(x=qld_renewables_df['MONTH'], y=qld_renewables_df['SCADAVALUE'], mode='lines+markers', name='QLD Renewables'))
fig = fig.add_trace(go.Scatter(x=vic_renewables_df['MONTH'], y=vic_renewables_df['SCADAVALUE'], mode='lines+markers', name='VIC Renewables'))
fig = fig.add_trace(go.Scatter(x=sa_renewables_df['MONTH'], y=sa_renewables_df['SCADAVALUE'], mode='lines+markers', name='SA Renewables'))
fig = fig.add_trace(go.Scatter(x=tas_renewables_df['MONTH'], y=tas_renewables_df['SCADAVALUE'], mode='lines+markers', name='TAS Renewables'))

fig.update_layout(title='Renewable Energy Generation by State (2018 - 2025)', xaxis_title='Month', yaxis_title='Generation (MWh)')
fig.show()

Generally, all states show growth in renewable generation, with most annual peaks around July while non-renewables are slowly declining in generation

In [65]:
fig = go.Figure()

fig = fig.add_trace(go.Scatter(x=nsw_non_renewables_df['MONTH'], y=nsw_non_renewables_df['SCADAVALUE'], mode='lines+markers', name='NSW Non-Renewables'))
fig = fig.add_trace(go.Scatter(x=qld_non_renewables_df['MONTH'], y=qld_non_renewables_df['SCADAVALUE'], mode='lines+markers', name='QLD Non-Renewables'))
fig = fig.add_trace(go.Scatter(x=vic_non_renewables_df['MONTH'], y=vic_non_renewables_df['SCADAVALUE'], mode='lines+markers', name='VIC Non-Renewables'))
fig = fig.add_trace(go.Scatter(x=sa_non_renewables_df['MONTH'], y=sa_non_renewables_df['SCADAVALUE'], mode='lines+markers', name='SA Non-Renewables'))
fig = fig.add_trace(go.Scatter(x=tas_non_renewables_df['MONTH'], y=tas_non_renewables_df['SCADAVALUE'], mode='lines+markers', name='TAS Non-Renewables'))

fig.update_layout(title='Non-Renewable Energy Generation by State (2018 - 2025)', xaxis_title='Month', yaxis_title='Generation (MWh)')
fig.show()

Finding where the negative values are in each state and what those fuel types are

In [66]:
nsw_neg_generation = nsw_data[nsw_data['SCADAVALUE'] < 0]
nsw_neg_generation['Fuel Type'].unique()

array(['Wind - Wind', 'Fossil - Diesel', 'Fossil - Natural Gas Pipeline',
       'Hydro - Water', nan, 'Solar - Solar',
       'Renewable Biomass or Waste - Bagasse', 'Other - Other',
       'Renewable Biomass or Waste - Biomass'], dtype=object)

In [67]:
tas_neg_generation = tas_data[tas_data['SCADAVALUE'] < 0]

tas_neg_generation['Fuel Type'].unique()

array(['Wind - Wind', 'Hydro - Water', 'Fossil - Natural Gas Pipeline'],
      dtype=object)

In [68]:
qld_neg_generation = qld_data[qld_data['SCADAVALUE'] < 0]
qld_neg_generation['Fuel Type'].unique()

array([nan, 'Renewable Biomass or Waste - Bagasse',
       'Fossil - Waste Coal Mine Gas', 'Hydro - Water', 'Solar - Solar',
       'Fossil - Natural Gas Pipeline', 'Fossil - Black Coal',
       'Fossil - Coal Seam Methane', 'Other - Other'], dtype=object)

In [69]:
vic_neg_generation = vic_data[vic_data['SCADAVALUE'] < 0]
vic_neg_generation['Fuel Type'].unique()

array(['Hydro - Water', nan, 'Wind - Wind',
       'Fossil - Natural Gas Pipeline', 'Solar - Solar',
       'Fossil - Brown Coal', 'Other - Other'], dtype=object)

In [70]:
sa_neg_generation = sa_data[sa_data['SCADAVALUE'] < 0]
sa_neg_generation['Fuel Type'].unique()

array(['Wind - Wind', 'Fossil - Diesel', nan, 'Solar - Solar',
       'Fossil - Natural Gas Pipeline', 'Other - Other'], dtype=object)

Looking at how often each of the fuel sources go negative

In [71]:
qld_data[(qld_data['SCADAVALUE'] < 0) & (qld_data['Fuel Type'] != 'Other - Other')].dropna().reset_index(drop = True)['Fuel Type'].value_counts()

Fuel Type
Solar - Solar                           626843
Renewable Biomass or Waste - Bagasse    545351
Hydro - Water                           137902
Fossil - Coal Seam Methane              107748
Fossil - Black Coal                      81990
Fossil - Waste Coal Mine Gas              5778
Fossil - Natural Gas Pipeline              921
Name: count, dtype: int64

In [72]:
nsw_data[(nsw_data['SCADAVALUE'] < 0) & (nsw_data['Fuel Type'] != 'Other - Other')].dropna().reset_index(drop = True)['Fuel Type'].value_counts()

Fuel Type
Fossil - Diesel                         269578
Wind - Wind                             213921
Solar - Solar                           160403
Renewable Biomass or Waste - Bagasse     64222
Hydro - Water                            40061
Renewable Biomass or Waste - Biomass     18904
Fossil - Natural Gas Pipeline             7130
Name: count, dtype: int64

In [73]:
tas_data[(tas_data['SCADAVALUE'] < 0) & (tas_data['Fuel Type'] != 'Other - Other')].dropna().reset_index(drop = True)['Fuel Type'].value_counts()

Fuel Type
Fossil - Natural Gas Pipeline    231658
Hydro - Water                    194842
Wind - Wind                       61894
Name: count, dtype: int64

In [74]:
vic_data[(vic_data['SCADAVALUE'] < 0) & (vic_data['Fuel Type'] != 'Other - Other')].dropna().reset_index(drop = True)['Fuel Type'].value_counts()

Fuel Type
Hydro - Water                    540482
Wind - Wind                      402426
Fossil - Natural Gas Pipeline     80906
Solar - Solar                      3249
Fossil - Brown Coal                   2
Name: count, dtype: int64

In [75]:
sa_data[(sa_data['SCADAVALUE'] < 0) & (sa_data['Fuel Type'] != 'Other - Other')].dropna().reset_index(drop = True)['Fuel Type'].value_counts()

Fuel Type
Wind - Wind                      571213
Fossil - Diesel                  225660
Fossil - Natural Gas Pipeline    129054
Solar - Solar                    101912
Name: count, dtype: int64

Checking what the lowest values are when the generators go negative

In [76]:
qld_data = qld_data.dropna(subset = ['Fuel Type']).reset_index(drop = True)
qld_data = qld_data[qld_data['Fuel Type'] != 'Other - Other']

nsw_data = nsw_data.dropna(subset = ['Fuel Type']).reset_index(drop = True)
nsw_data = nsw_data[nsw_data['Fuel Type'] != 'Other - Other']

tas_data = tas_data.dropna(subset = ['Fuel Type']).reset_index(drop = True)
tas_data = tas_data[tas_data['Fuel Type'] != 'Other - Other']

vic_data = vic_data.dropna(subset = ['Fuel Type']).reset_index(drop = True)
vic_data = vic_data[vic_data['Fuel Type'] != 'Other - Other']

sa_data = sa_data.dropna(subset = ['Fuel Type']).reset_index(drop = True)
sa_data = sa_data[sa_data['Fuel Type'] != 'Other - Other']

In [77]:
def print_lowest_scada_values(state_data, state):

    print('State Data: ', state)
    for fuel_type in state_data['Fuel Type'].unique():
        lowest_val = state_data[state_data['Fuel Type'] == fuel_type]['SCADAVALUE'].min()
        if lowest_val < 0:
            print(f'The lowest values for {fuel_type} in {state} was found to be {lowest_val}')

    print('\n\n')
    return 

### Adding the Dispatch Price Data with the State Level Datasets

Function used to merge the data. The following function will do the following:
For each of the pricing folders:
1. Access the files within the directory
2. Read in the csv files, only taking the `SETTLEMENTDATE`, `REGIONID` and `RRP`, dropping any NA values from those columns
3. Convert the data type of `SETTLEMENTDATE` to datetime
4. Check there is an even number of each region in the pricing dataframe
5. Pd.merge the pricing data with the 5 min data

It also takes in the 5 min data of the state

In [ ]:
def format_dataframe_report(df, col, date_column, duid):
    start_formatting = time.time()
    if date_column:
        df[col] = pd.to_datetime(df[col], format = '%Y/%m/%d %H:%M:%S')
        df['MONTH'] = df[col].dt.to_period('M')

    if duid:
        df['DUID'] = df['DUID'].astype('category')

    end_formatting = time.time()
    print(f"Formatting the {col} column took {end_formatting - start_formatting} seconds.")
    return df

def merge_price_data(state_data_list):
    pricing_dict = {}

    print(state_data_list['REGIONID'].unique())
    
    pricing_folders = [
        os.path.join(current_dir, 'data', 'price', 'public'),
        os.path.join(current_dir, 'data', 'price', 'archived')
    ]

    for dir in pricing_folders:
        # monthly_price_files = sorted(os.listdir(dir))


        print('Looking into directory: ', dir)

    #     for file in monthly_price_files:
    #         file_path = os.path.join(dir, file)
    #         print('Processing File: ', file)
            

    #         monthly_prices = log_time("Read File", read_file, file_path, ['SETTLEMENTDATE', 'REGIONID', 'RRP'])
            
    #         start_formatting = time.time()
    #         monthly_prices['SETTLEMENTDATE'] = pd.to_datetime(monthly_prices['SETTLEMENTDATE'])
    #         state_data['SETTLEMENTDATE'] = pd.to_datetime(state_data['SETTLEMENTDATE'])
    #         end_formatting = time.time()
    #         print(f"Formatting the SETTLEMENTDATE column took {round(end_formatting - start_formatting, 2)} seconds.")

    #         state_monthly_prices = monthly_prices[monthly_prices['REGIONID'] == region_id]

    #         state_rrps = state_monthly_prices.set_index('SETTLEMENTDATE')['RRP'].dropna().to_dict()
    #         print(f'State RRPs', state_rrps, '\n\n')
    #         pricing_dict.update(state_rrps)

    # state_data['RRP'] = state_data['SETTLEMENTDATE'].map(pricing_dict)

    # state_data['SETTLEMENTDATE'] = state_data['SETTLEMENTDATE'].dt.strftime('%Y-%m-%d %H:%M:%S')

    return state_data_list

In [81]:
for dataset in nem_datasets:
    print(dataset['REGIONID'].unique())

['NSW1']
['QLD1']
['SA1']
['TAS1']
['VIC1']


In [79]:
merge_price_data(qld_data)


['QLD1']
Looking into directory:  /Users/aidanlockwood/Documents/GitHub/IFN695-Codebase/data/price/public
Looking into directory:  /Users/aidanlockwood/Documents/GitHub/IFN695-Codebase/data/price/archived


,SETTLEMENTDATE,DUID,SCADAVALUE,MONTH,REGIONID,Fuel Type
0,2018-01-01 00:05:00,BARCSF1,1.300000,2018-01,QLD1,Solar - Solar
1,2018-01-01 00:05:00,GERMCRK,33.200001,2018-01,QLD1,Fossil - Waste Coal Mine Gas
2,2018-01-01 00:05:00,MBAHNTH,51.618744,2018-01,QLD1,Fossil - Waste Coal Mine Gas
3,2018-01-01 00:05:00,RPCG,10.400000,2018-01,QLD1,Renewable Biomass or Waste - Bagasse
4,2018-01-01 00:10:00,BARCSF1,1.300000,2018-01,QLD1,Solar - Solar
...,...,...,...,...,...,...
67582074,2025-07-31 23:55:00,WOOLGSF1,0.000000,2025-07,QLD1,Solar - Solar
67582075,2025-07-31 23:55:00,YABULU,160.041490,2025-07,QLD1,Fossil - Natural Gas Pipeline
67582076,2025-07-31 23:55:00,YABULU2,0.000000,2025-07,QLD1,Fossil - Natural Gas Pipeline
67582077,2025-07-31 23:55:00,YARANSF1,0.000000,2025-07,QLD1,Solar - Solar
